#  Scoring function

In [102]:
def project_1_scoring(data):
    
   
    from copy import deepcopy
    from sklearn.linear_model import LogisticRegression
    import category_encoders as ce
    import pickle
    import numpy as np
    
    X = data.copy()
    
    '''Load Artifacts'''
    artifacts_dict_file = open("artifacts_dictnew_file.pkl", "rb")
    artifacts_dict = pickle.load(file=artifacts_dict_file)
    artifacts_dict_file.close()
    
    model = artifacts_dict["model"],
    woe_cols = artifacts_dict["woe_cols"]
    woe_encoder = artifacts_dict["woe_encoder"]
    ohe_cols = artifacts_dict["ohe_columns"]
    drop_from_data = artifacts_dict["drop_from_data"]
    col_remove_dollar = artifacts_dict["col_remove_dollar"]
    threshold = artifacts_dict["threshold"]
    estimator = artifacts_dict["estimator"]
    original_columns= artifacts_dict["original_columns"]
    columns_to_score=artifacts_dict["columns_to_score"]
    
   
    fill_na={}
    for col in X.columns:
        if pd.api.types.is_numeric_dtype(X[col].dtype):
            fill_na[col]=0
        else:
            fill_na[col]='Missing'
    X.fillna(value=fill_na,inplace=True)
    
    # Remove all the columns that were unseen during training
    X=X[original_columns]
    
    # Remove Dollars from numerical vars
    for col in col_remove_dollar:
        X[col] = X[col].str.replace('$','',regex=True).replace(',','',regex=True)
        X[col] = X[col].astype(float)
    
    # Adding New Features
    X['NAICS'] = X['NAICS']%100
    X['NAICS'] = X['NAICS'].astype('category',copy=False)
    
    X['Net_Loan']=X['DisbursementGross']-X['BalanceGross']
    X['SBA_Payable']=(X['SBA_Appv']/X['GrAppv'])
    # Dimension Reduction
    X['Term']=X['Term']/12
    
    values=['Y','N']
    X = X[X.LowDoc.isin(values) == True]
    X['LowDoc']=X['LowDoc'].astype('category',copy=True)
    
    X['NewExist']=X['NewExist'].astype('category',copy=True)
    X = X.loc[X['NewExist'] != 0]
    
    X['UrbanRural']=X['UrbanRural'].apply(lambda x: 'Urban' if x == 1 else  'Rural' if x==2 else 'Undefined')
    X['UrbanRural']=X['UrbanRural'].astype('category',copy=True)
    
    X['FranchiseCode']=X['FranchiseCode'].apply(lambda x: 'N' if x in [0,1] else 'Y')
    X['FranchiseCode'] = X['FranchiseCode'].astype('category',copy=False)
    

    #one hot encoding
    for col in ohe_cols:
        one_hot_enc_X=pd.get_dummies(X[col], prefix=col)
        X = pd.concat([X, one_hot_enc_X], axis=1)
    
    # WOE Encoding        
    woe_enc_test = woe_encoder.transform(X[woe_cols])
    for col in woe_cols:
        X[col+"_woe"] = woe_enc_test[col]
    
      
    # logistic regression predict
    X_test=X[columns_to_score]
    model=artifacts_dict['model']
    y_pred_proba = model.predict_proba(X_test)
    y_pred = (y_pred_proba[:,0] < threshold).astype(np.int16)
    
    d = {#"index":data["MIS_Status"],
         "label":y_pred,
         "probability_0":y_pred_proba[:,0],
         "probability_1":y_pred_proba[:,1]}
    
    return pd.DataFrame(d)
    

In [103]:
''' The below code was used for testing the above scoring function '''

#import pandas as pd
#loan = pd.read_csv("test_data.csv")
#loan1=loan.drop(columns=['MIS_Status']).head(10000)
#project_1_scoring(loan1)

,label,probability_0,probability_1
0,0,0.999267,0.000733
1,0,0.995578,0.004422
2,0,0.895365,0.104635
3,0,0.933580,0.066420
4,0,0.859961,0.140039
...,...,...,...
6008,0,0.769198,0.230802
6009,0,0.976354,0.023646
6010,0,0.760009,0.239991
6011,0,0.949892,0.050108
